# 9장 컴퓨터 비전을 위한 고급 딥러닝 1부

**감사말**: 프랑소와 숄레의 [Deep Learning with Python, Second Edition](https://www.manning.com/books/deep-learning-with-python-second-edition?a_aid=keras&a_bid=76564dff) 9장에 사용된 코드에 대한 설명을 담고 있으며 텐서플로우 2.6 버전에서 작성되었습니다. 소스코드를 공개한 저자에게 감사드립니다.

**tensorflow 버전과 GPU 확인**
- 구글 코랩 설정: '런타임 -> 런타임 유형 변경' 메뉴에서 GPU 지정 후 아래 명령어 실행 결과 확인

    ```
    !nvidia-smi
    ```

- 사용되는 tensorflow 버전 확인

    ```python
    import tensorflow as tf
    tf.__version__
    ```
- tensorflow가 GPU를 사용하는지 여부 확인

    ```python
    tf.config.list_physical_devices('GPU')
    ```

## 주요 내용

- 합성곱 신경망의 주요 활용 분야(컴퓨터 비전)
    - 이미지 분류
    - 이미지 분할
    - 객체 탐지
- 합성곱 신경망의 기본 아키텍처
    - 잔차 연결(residual connections)
    - 배치 정규화(batch normalization)
    - 채널 분리 합성곱(depthwise separable convolutions)

## 9.1 컴퓨터 비전 분야 세 가지 주요 과제

컴퓨터 비전 분야에서 가장 주요한 연구 주제는 다음 세 가지이다.

- 이미지 분류(image classification): 이미지에 포함된 사물(들)의 클래스 분류
    - 단일 레이블 분류(single-label classification)
        - 예제: 고양이-강아지 분류 
    - 다중 레이블 분류(multi-label classification)
        - 예제: 구글 포토. 2만 개의 서로 다른 클래스 사용
- 이미지 분할(image segmentation): 이미지를 특정 클래스를 포함하는 영역으로 분할
    - 예제: 줌(Zoom), 구글 미트(Google Meet) 등에서 사용되는 배경 블러처리 기능
- 객체 탐지(object detection): 이미지에 포함된 객체 주의에 경계상자(bounding box) 그리기
    - 예제: 자율주행 자동차의 주변에 위치한 다른 자동차, 행인, 신호등 등 탐지 기능

<div align="center"><img src="https://drek4537l1klr.cloudfront.net/chollet2/v-7/Figures/computer_vision_tasks.png" style="width:100%;"></div>

그림 출처: [Deep Learning with Python(Manning MEAP)](https://www.manning.com/books/deep-learning-with-python-second-edition)

언급된 3 분야 이외에 아래 컴퓨터 비전 분야에서도 딥러닝이 중요하게 활용된다.

- 이미지 유사도 측정(image similarity scoring),
- 키포인트 탐지(keypoint detection),
- 자세 추정(pose estimation),
- 3D 메쉬 추정(3D mesh estimation), 등등

하지만 객체 탐지를 포함해서 언급된 분야 모두
기초 수준을 넘어서기에 여기서는 다루지 않는다.
다만 객체 탐지 관련해서 다음 논문을 참고할 것을 권유한다.

- 객체 탐지 참고 자료: [RetinaNet 활용 객체 탐지](https://keras.io/examples/vision/retinanet/)

아래에서는 이미지 분할을 예제를 활용하여 좀 더 상세하게 설명한다.

## 9.2 이미지 분할 예제

### 이미지 분할 방식

이미지 분할은 크게 두 종류 방식을 사용한다.

- 시맨틱 분할(semantic segmentation): 클래스 별 분할.
    - 아래 사진 왼편: 배경과 구분된 고양이들을 묶어서 cat 클래스로 구별하지 않고 분류.
- 인스턴스 분할(instance segmentation): 클래스 및 객체 별 분할.
    - 아래 사진 오른편: 배경과 구분된 각각의 고양이를 cat1, cat2 등으로 구별해서 분류.

<div align="center"><img src="https://drek4537l1klr.cloudfront.net/chollet2/v-7/Figures/instance_segmentation.png" style="width:100%;"></div>

그림 출처: [Deep Learning with Python(Manning MEAP)](https://www.manning.com/books/deep-learning-with-python-second-edition)

여기서는 고양이와 강아지 사진을 이용하여 시맨틱 분할을 상세히 살펴본다.

### Oxford-IIIT 애완동물 데이터셋

[Oxford-IIIT 애완동물 데이터셋](https://www.robots.ox.ac.uk/~vgg/data/pets/)은
강아지와 고양이를 비롯해서 37종의 애완동물의 다양한 크기와 다양한 자세를 담은
7,390장의 사진으로 구성된다.

- 데이터셋 크기: 7,390
- 총 클래스 수: 37
- 클래스 별 사진 수: 약 200 장
- 사진 별 레이블: 종과 품종, 머리 표시 경계상자, 트라이맵 분할(trimap segmentation)

<div align="center"><img src="https://www.robots.ox.ac.uk/~vgg/data/pets/pet_annotations.jpg" style="width:100%;"></div>

그림 출처: [Oxford-IIIT 애완동물 데이터셋](https://www.robots.ox.ac.uk/~vgg/data/pets/)

**데이터셋 다운로드**

In [0]:
!wget http://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz
!wget http://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz
!tar -xf images.tar.gz
!tar -xf annotations.tar.gz

In [0]:
import os

input_dir = "images/"
target_dir = "annotations/trimaps/"

input_img_paths = sorted(
    [os.path.join(input_dir, fname)
     for fname in os.listdir(input_dir)
     if fname.endswith(".jpg")])
target_paths = sorted(
    [os.path.join(target_dir, fname)
     for fname in os.listdir(target_dir)
     if fname.endswith(".png") and not fname.startswith(".")])

In [0]:
import matplotlib.pyplot as plt
from tensorflow.keras.utils import load_img, img_to_array

plt.axis("off")
plt.imshow(load_img(input_img_paths[9]))

In [0]:
def display_target(target_array):
    normalized_array = (target_array.astype("uint8") - 1) * 127
    plt.axis("off")
    plt.imshow(normalized_array[:, :, 0])

img = img_to_array(load_img(target_paths[9], color_mode="grayscale"))
display_target(img)

In [0]:
import numpy as np
import random

img_size = (200, 200)
num_imgs = len(input_img_paths)

random.Random(1337).shuffle(input_img_paths)
random.Random(1337).shuffle(target_paths)

def path_to_input_image(path):
    return img_to_array(load_img(path, target_size=img_size))

def path_to_target(path):
    img = img_to_array(
        load_img(path, target_size=img_size, color_mode="grayscale"))
    img = img.astype("uint8") - 1
    return img

input_imgs = np.zeros((num_imgs,) + img_size + (3,), dtype="float32")
targets = np.zeros((num_imgs,) + img_size + (1,), dtype="uint8")
for i in range(num_imgs):
    input_imgs[i] = path_to_input_image(input_img_paths[i])
    targets[i] = path_to_target(target_paths[i])

num_val_samples = 1000
train_input_imgs = input_imgs[:-num_val_samples]
train_targets = targets[:-num_val_samples]
val_input_imgs = input_imgs[-num_val_samples:]
val_targets = targets[-num_val_samples:]

In [0]:
from tensorflow import keras
from tensorflow.keras import layers

def get_model(img_size, num_classes):
    inputs = keras.Input(shape=img_size + (3,))
    x = layers.Rescaling(1./255)(inputs)

    x = layers.Conv2D(64, 3, strides=2, activation="relu", padding="same")(x)
    x = layers.Conv2D(64, 3, activation="relu", padding="same")(x)
    x = layers.Conv2D(128, 3, strides=2, activation="relu", padding="same")(x)
    x = layers.Conv2D(128, 3, activation="relu", padding="same")(x)
    x = layers.Conv2D(256, 3, strides=2, padding="same", activation="relu")(x)
    x = layers.Conv2D(256, 3, activation="relu", padding="same")(x)

    x = layers.Conv2DTranspose(256, 3, activation="relu", padding="same")(x)
    x = layers.Conv2DTranspose(256, 3, activation="relu", padding="same", strides=2)(x)
    x = layers.Conv2DTranspose(128, 3, activation="relu", padding="same")(x)
    x = layers.Conv2DTranspose(128, 3, activation="relu", padding="same", strides=2)(x)
    x = layers.Conv2DTranspose(64, 3, activation="relu", padding="same")(x)
    x = layers.Conv2DTranspose(64, 3, activation="relu", padding="same", strides=2)(x)

    outputs = layers.Conv2D(num_classes, 3, activation="softmax", padding="same")(x)

    model = keras.Model(inputs, outputs)
    return model

model = get_model(img_size=img_size, num_classes=3)
model.summary()

In [0]:
model.compile(optimizer="rmsprop", loss="sparse_categorical_crossentropy")

callbacks = [
    keras.callbacks.ModelCheckpoint("oxford_segmentation.keras",
                                    save_best_only=True)
]

history = model.fit(train_input_imgs, train_targets,
                    epochs=50,
                    callbacks=callbacks,
                    batch_size=64,
                    validation_data=(val_input_imgs, val_targets))

In [0]:
epochs = range(1, len(history.history["loss"]) + 1)
loss = history.history["loss"]
val_loss = history.history["val_loss"]
plt.figure()
plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.legend()

In [0]:
from tensorflow.keras.utils import array_to_img

model = keras.models.load_model("oxford_segmentation.keras")

i = 4
test_image = val_input_imgs[i]
plt.axis("off")
plt.imshow(array_to_img(test_image))

mask = model.predict(np.expand_dims(test_image, 0))[0]

def display_mask(pred):
    mask = np.argmax(pred, axis=-1)
    mask *= 127
    plt.axis("off")
    plt.imshow(mask)

display_mask(mask)